In [1]:
import os
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [26]:
import tensorflow as tf
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.8 and logs['val_accuracy'] > 0.6:
            print("\nReached desired accuracy, stopping training.")
            self.model.stop_training = True

In [19]:
# target_size = (128, 128)
batch_size = 32
num_classes = 42  # Adjust based on your number of classes

In [4]:
data_directory = 'datasets_character_anime'
image_filenames = [os.path.join(data_directory, folder, filename) for folder in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, folder)) for filename in os.listdir(os.path.join(data_directory, folder)) if filename.endswith(('.jpg', '.png', '.jpeg'))]


In [5]:
for filename in image_filenames:
    img = Image.open(filename)
    img = img.resize((128, 128))
    img.save(filename)

In [9]:
datagen = ImageDataGenerator(
        rescale=1./255,  # normalize pixel values to be between 0 and 1
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.1
    )

# datagen = ImageDataGenerator(
#     rescale=1./255,
#     validation_split=0.2,
# )


train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True  # Add shuffle parameter
)

validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # No need to shuffle validation data
)

Found 4030 images belonging to 42 classes.
Found 426 images belonging to 42 classes.


In [10]:
print(train_generator.image_shape)
print(validation_generator.image_shape)


(128, 128, 3)
(128, 128, 3)


In [20]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))  # Input shape is (64, 64, 1) for grayscale images
model.add(MaxPooling2D())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))  # Add dropout after the Flatten layer or between Dense layers
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

In [69]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))


In [21]:
from keras.optimizers import Adam

custom_optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
num_epochs = 100
custom_callback = CustomCallback()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[custom_callback],
    epochs=num_epochs
)

Epoch 1/100
125/125 [==============================] - ETA: 0s - loss: 0.6470 - accuracy: 0.8034
Reached desired accuracy, stopping training.
125/125 [==============================] - 14s 113ms/step - loss: 0.6470 - accuracy: 0.8034 - val_loss: 1.7701 - val_accuracy: 0.6106


In [33]:
model.save('new_anime_classification2.h5')

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = 'new_anime_classification.h5'  # Replace with the actual path to your model
loaded_model = load_model(model_path)

# Function to preprocess the input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

# Function to make predictions
def predict_image(model, img_path):
    processed_image = preprocess_image(img_path)
    prediction = model.predict(processed_image)
    return prediction

# Replace 'your_image.jpg' with the actual path to the image you want to test
image_path = 'uploads/2.png'
result = predict_image(loaded_model, image_path)
# Assuming you have 36 classes, replace this with your actual class names
class_names = ['Anya_Forger', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'Gojo_Satoru', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'amelia_watson', 'arima_kousei', 'gawr_gura', 'hoshino_ai', 'hutao_genshin', 'mikasa', 'tanjiro', 'violet_evergarden', 'zeta_hololive']

# Get the predicted class index
predicted_class_index = np.argmax(result)
predicted_class = class_names[predicted_class_index]

print(f"The predicted class is: {predicted_class}")

1/1 [==============================] - 0s 86ms/step
The predicted class is: Anya_Forger


In [28]:
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

Class names: ['Anya_Forger', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'Gojo_Satoru', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'amelia_watson', 'arima_kousei', 'gawr_gura', 'hoshino_ai', 'hutao_genshin', 'mikasa', 'tanjiro', 'violet_evergarden', 'zeta_hololive']


In [32]:
import tensorflow as tf

# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('anime_classification_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ACER\AppData\Local\Temp\tmpc8gcyd68\assets


INFO:tensorflow:Assets written to: C:\Users\ACER\AppData\Local\Temp\tmpc8gcyd68\assets
